# Tutoriel #3 - Geocoding, geonaming

Ce notebook est une introduction aux :

1. **Gecoding** : action qui consiste à convertir des adresses en coordonnées géographiques à placer sur une carte.
2. **Geonaming** : action qui consiste à donner un nom à un lieu sur une carte.

## Mise en place
Commençons par charger la librairie `Pandas`.

In [ ]:
import pandas as pd

## Données initiales

Nous ne disposons, ici, que des codes géographiques des départements. Ces codes sont appelés **codes officiels géographiques** (COG).

Les départements métropolitains étaient numérotés dans l'ordre alphabétique à l'origine (01 Ain, 02 Aisne, 03 Allier, 04 Alpes-de-Haute-Provence, 12 Aveyron, 13 Bouches-du-Rhône, ..., 93 Seine-Saint-Denis, 94 Val-de-Marne, etc.), avant des changements de noms et le redécoupage de l'Île-de-France en 1968.

En 1976, la Corse (20) est divisée en deux. Plutôt que de revoir la numérotation globale, les deux nouveaux départements reçoivent un code alphanumérique : 2A (comme Ajaccio) pour la Corse-du-Sud, 2B (comme Bastia) pour la Haute-Corse.

> En 1946, les départements d'outre-mer sont créés et reçoivent en 1948 un code avec le préfixe 97 : Guadeloupe 971,  Martinique 972, Guyane 973, La Réunion 974.

**Attention** : Seuls les départements métropolitains seront traités dans ce tutoriel. Les COG pris en compte seront donc ceux allant de `01` à `95`.

In [ ]:
# Génération des COG métropolitains de 01 à 95 codés sur 2 caractères
cog = [ f"{code:02d}" for code in range(1,96)]

# On remplace le code 20 (Corse) par les codes des deux départements de la Corse, soit 2A et 2B
cog = [x for item in cog for x in (['2A', '2B'] if item == '20' else [item])]
print(cog)

## Geocoding et geonaming des départements

Afin de déterminer **le nom et les limites géographiques de chaque département**, à partir de la liste des COG, nous allons faire appel à `Nominatim` un des services de géo-information présents sur le Web.

> **Nominatim** peut effectuer un géocodage direct et inverse et prend en charge tous les pays, langues et formats d'adresses du monde. Il est développé et maintenu par la communauté OpenStreetMap (OSM) et utilisé comme service de géocodage par défaut pour les données OSM.

Par défaut, Nominatim renvoie une page HTML formatée, prête à être affichée dans le navigateur.
> Essayez en cliquant sur ce lien https://nominatim.openstreetmap.org/ui/search.html?county=FR-94 qui permet de géocoder le département 94 (Val-de-Marne).

Si vous avez besoin d'un format de réponse lisible par une machine, vous devrez modifier l'URL de la manière suivante : https://nominatim.openstreetmap.org/ui/search et ajouter les paramètres :
*  `format` et le définir sur l'un des formats de sortie «xml», «json», «geojson» et «geocodejson».
*  `polygon_geojson=1` afin d'obtenir les limites du département sous forme vectorielle.

Nous recommandons d'utiliser «json» ou «geojson». Le deuxième format «geojson», est généralement compatible avec la spécification GeoJSON et qui, dans le cas de Nominatim, inclut la structure d'adresse la plus complète et la plus standardisée.

L'API est interrogeable à l'adresse https://nominatim.openstreetmap.org/

Exemple de sortie de géocodage pour "/search?state=France&county=94&format=json&polygon_geojson=1&limit=1"

In [ ]:
import pandas as pd

# Requête vers l'API Nominatim
code_dpt = 94
URL = f"https://nominatim.openstreetmap.org/search?state=France&county=FR-{code_dpt}&format=json&polygon_geojson=1&limit=1"
# Exécution de la requête à l'aide de Pandas
data = pd.read_json(URL)
data.head()

On remarque que le DataFrame ne contient qu'une seule ligne (une seule série à l'index 0). Le code suivant permet de récupérer cette série.

In [ ]:
# Extraction de la série du DataFrame
data = data.iloc[0]
print(data)

In [ ]:
# Geonaming : extraction du champ 'display_name'
nom = data.display_name
print(nom)

In [ ]:
# Geocoding : extraction des coordonnées centrales (centroïde du département)
lat, lon = data.lat, data.lon
print(f"latitude={lat}, longitude={lon}")

In [ ]:
# Geocoding : extraction de la limite du département au format geojson
limite = data.geojson
print(limite)

## Web mapping

Nous disposons, à présent, de toutes les informations qui vont nous permettre de représenter le département sur une carte.

La carte est construite à partir du **service de cartes (WMS)** offert par `Folium`.

Sur cette carte, les limites du département sont placées, ainsi que le nom sous forme d'une info-bulle avec `Tooltip`.


Si  besoin, installer le module folium à l'aide de la commande
```shell
pip install folium
```

In [ ]:
import folium

# Service WMS : récupérartion de la carte centrée sur le centroïde du département
m = folium.Map(location=[lat, lon], zoom_start=11)

# Ajouter le nom et les contours du département au format GeoJSON à la carte
folium.GeoJson(limite, tooltip=folium.Tooltip(nom)).add_to(m)

# Afficher la carte dans le notebook
m

## Utilisation d'un géocodeur intégré

Il est possible d'accéder à `Nominatim` sans passer par l'API mais en utilisant le module Python `GeoPy`.

Nous allons voir de quelle manière utiliser les fonctionalités de géocodage de ce module.

Commençons par installer `GeoPy` :

In [ ]:
! pip install geopy

A présent, nous allons tester le géocodage d'un département en donnant uniquement son COG, ici 2A pour la Corse-du-Sud.

> Remarque : lors de la requête de géocodage le préfix `FR-` est ajouté devant le COG afin de lever toute ambiguïté sur le sens à donner à ce code.

In [ ]:
from geopy.geocoders import Nominatim

# Initialiser le géocodeur Nominatim
geolocator = Nominatim(user_agent="geo_parsing_example")

# Recherche d'un département par son code, ici la Corse-du-Sud (2A)
code_dpt = '2A'
location = geolocator.geocode(f"FR-{code_dpt}", geometry="geojson", language ='fr', addressdetails = True, timeout=1)

# 'FR-' est ajouté en préfix du COG du département : soit 'FR-2A' ici.
# geometry="geojson" : permet de récupérer les limites du département.
# language ='fr' : permet de spécifier la langue utilisée pour les résultats renvoyés.
# addressdetails = True : permet d'assurer de récuperer tous les détails concernant le lieu.
# timeout = 1 : temporisation (en secondes) pour attendre la réponse du serveur.

# Affichage des informations reçues
location.raw

Le champ `raw` de la variable contient un dictionnaire qui renferme l'ensmble des information retournée par Nominatim.

Il reste à afficher les données récupérées sur une carte :

In [ ]:
import folium

# Position du centroïde
lat = location.raw['lat']
lon = location.raw['lon']

# Limite du département
limite = location.raw['geojson']

# Les noms du département, de la région et du pays
nom = location.raw['name']
region = location.raw['address']['state']
pays = location.raw['address']['country']

# Créer la carte centrée sur le centroïde du département
m = folium.Map(location=[lat, lon], zoom_start=8)

# Ajouter le nom et les contours du département au format GeoJSON à la carte
folium.GeoJson(limite, tooltip=folium.Tooltip(f"{nom}, {region}, {pays}")).add_to(m)

# Afficher la carte dans le notebook
m

On sauvegarde la carte au format HTML :

In [ ]:
m.save(f"carte_departement_{code_dpt}.html")

> La page web est créer dans le dossier courant. Vous pouvez l'ouvrir, au sein d'un serveur local, en la sélectionnant et en cliquant sur "Go Live" en bas à droite de l'interface VSC.

## A faire ...

En utilisant l'approche de géocodage par le géocodeur intégré à `GeoPy`, vous devez construire une carte regroupant tous les départements métropolitains en partant uniqement de la liste des COG (voir première partie du tutoriel).

La carte devra :
* être centrée sur le centroïde de la France métropolitaine ;
* avoir un niveau de zoom adapté afin de visualiser l'ensemble du territoire.

<br>

> Cette carte des départements pourra être sauvegardée au format HTML et ouverte dans un navigateur Web depuis un serveur local.


In [ ]:
# Code du GeoMapping de tous les départements métropolitains
'''à compléter...'''